In [ ]:
import tensorflow.compat.v1 as tf

In [6]:
export_dir = './saved'
graph_pb = 'model/har_lstm/frozen_model.pb'

In [7]:
# Load the TensorFlow v1 model from .pb file
with tf.Graph().as_default():
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(graph_pb, 'rb') as f:
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    # Print the tensor names in the graph
    with tf.Session() as sess:
        # Get all the tensor names
        tensor_names = [tensor.name for tensor in tf.get_default_graph().as_graph_def().node]

        # Print the tensor names
        for name in tensor_names:
            print(name)


Variable
Variable/read
Variable_1
Variable_1/read
Variable_2
Variable_2/read
Variable_3
Variable_3/read
inputs
transpose/perm
transpose
Reshape/shape
Reshape
MatMul
add
Relu
split/split_dim
split
rnn/Shape
rnn/strided_slice/stack
rnn/strided_slice/stack_1
rnn/strided_slice/stack_2
rnn/strided_slice
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/stack/1
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/stack
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros/Const
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/stack_1/1
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/stack_1
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1/Const
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState_1/stack/1
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState_1/stack
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros/Const
rnn/MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros


In [5]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.saved_model import signature_constants
from tensorflow.compat.v1.saved_model import signature_def_utils

# Load the TensorFlow v1 model from .pb file
with tf.Graph().as_default():
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(graph_pb, 'rb') as f:
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    # Define the input and output tensor names
    input_tensor_name = 'input'
    output_tensor_name = 'y_'

    # Create a session and initialize variables
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Define the inputs and outputs for the SavedModel
        inputs = {input_tensor_name: sess.graph.get_tensor_by_name(input_tensor_name)}
        outputs = {output_tensor_name: sess.graph.get_tensor_by_name(output_tensor_name)}

        # Define the signature for the SavedModel
        signature = signature_def_utils.build_signature_def(
            inputs=inputs,
            outputs=outputs,
            method_name=signature_constants.PREDICT_METHOD_NAME
        )

        # Save the model as a SavedModel
        builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
        builder.add_meta_graph_and_variables(
            sess,
            [tf.saved_model.tag_constants.SERVING],
            signature_def_map={signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature}
        )
        builder.save()


2023-05-26 00:21:30.945923: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: The name 'input' refers to an Operation, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

In [8]:
# Read the contents of the .pbtxt file
with open('./model/har_lstm/checkpoint/har.pbtxt', 'r') as f:
    pbtxt = f.read()

# Parse the .pbtxt contents to retrieve the input and output tensor information
graph_def = tf.compat.v1.GraphDef()
tf.compat.v1.train.import_meta_graph(pbtxt, graph_def)

# Get the input and output tensor information
input_tensor_name = graph_def.node[0].name
input_tensor_shape = graph_def.node[0].attr['shape'].shape

output_tensor_name = graph_def.node[-1].name  # Assuming the last node is the output tensor

print("Input Tensor Name:", input_tensor_name)
print("Input Tensor Shape:", input_tensor_shape)
print("Output Tensor Name:", output_tensor_name)


RuntimeError: Exporting/importing meta graphs is not supported when eager execution is enabled. No graph exists when eager execution is enabled.